# PP correction
## Juncal Norte - august, 2022
#### Paul Sandoval Quilodrán - https://github.com/SQPaul

In [1]:
from osgeo import gdal
import pandas as pd
import rasterio as rio
from rasterio.plot import show
import numpy as np
import xarray as xr
import cftime

#### Generate net dem

In [32]:
#Read dem
dem_tif = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\dem500.tif")
dem = dem_tif.read(1)

for i in range(dem.shape[0]):
    for j in range(dem.shape[1]):
        dem[i,j] = int(dem[i,j])-2250 
        
net_dem = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\net_dem.tif",
                   "w",
                   driver = "GTiff",
                   height = dem.shape[0],
                   width = dem.shape[1],
                   count = 1,
                   nodata = np.nan,
                   dtype = np.float32,
                   crs = 32719,
                   transform = dem_tif.transform)

net_dem.write(dem,1)
net_dem.close()

7426


#### Generate PP

In [3]:
#Read PP
pp = rio.open(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_PR_JUNCALNORTE_500m.nc")
pp_cdf = xr.open_dataset(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_PR_JUNCALNORTE_500m.nc") 

#Read net dem
netdem_t = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\net_dem.tif")
netdem = netdem_t.read(1)

#Read mask
mask_tif = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\juncalnorte_mask_v0.tif") #windows
mask_array = mask_tif.read(1)

f1 = 1.53 #adjustment factor reanalisis product and observed data
f2 = 0.23 #El Yeso - CR2 adjustment
g_pp = 0.0008 #PP gradient per meter


for i in range(len(pp_cdf["pr"])):
    ii = i+1
    if ii < 10000:
        value = ii/1000
        f_value = "{:.3f}".format(value)
        #name = "/home/phi/Desktop/Projects/Juncal_norte/Climate_forcing/pp/pp00000"+str(f_value) #linux
        name = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp50000"+str(f_value) #windows
    else:
        value = ii/1000
        f_value = "{:.3f}".format(value)
        #name = "/home/phi/Desktop/Projects/Juncal_norte/Climate_forcing/pp/pp0000"+str(f_value) #linux
        name = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp5000"+str(f_value) #windows
    print(name)
    pp_it = pp.read(ii)
    for x in range(pp_it.shape[0]):
        for y in range(pp_it.shape[1]):
            pp_it[x,y] = (f1-f2+g_pp*(netdem[x,y]))*pp_it[x,y]
            if pp_it[x,y] <= 0:
                pp_it[x,y] = 0
            else:
                pp_it[x,y] = pp_it[x,y]
    maps = rio.open(
    str(name),"w",
    driver = "PCRaster",
    pcraster_valuescale = "VS_SCALAR",
    height = mask_tif.shape[0],
    width = mask_tif.shape[1],
    count = 1,
    nodata = mask_tif.nodata,
    dtype = np.float32,
    crs = 32719,
    transform = mask_tif.transform)
    maps.write(pp_it,1)
    maps.close()
    print(i)

C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.001
0
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.002
1
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.003
2
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.004
3
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.005
4
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.006
5
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.007
6
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.008
7
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.009
8
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.010
9
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.011
10
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp5\pp500000.012
11
C:\Users\pauls\Desktop\Pro